<a href="https://colab.research.google.com/github/pawansharmaaaa/Wav2Lip_Mod/blob/master/GFPGAN_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install basicsr facexlib realesrgan gfpgan

In [ ]:
from gfpgan import GFPGANer
import realesrgan
import cv2
import torch
from basicsr.utils import imwrite
import numpy as np

arch = 'clean'
channel_multiplier = 2
model_name = 'GFPGANv1.4'
url = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth'

restorer = GFPGANer(
    model_path = url,
    upscale = 2,
    arch = 'clean',
    channel_multiplier=2,
    bg_upsampler=None
)

im_path = "/content/10045.png"
input_img = cv2.imread(im_path, cv2.IMREAD_COLOR)

c_faces, res_faces, res_imgs = restorer.enhance(
    input_img,
    has_aligned=True,
    only_center_face=True,
    paste_back=True,
    weight = 0.6
)

res = np.array(res_imgs)
print(res)
cv2.imwrite('/out.png', res)

#TRIAL AND ERROR

In [ ]:
!git clone https://github.com/TencentARC/GFPGAN.git
!cd GFPGAN

In [ ]:
!pip install basicsr facexlib

In [ ]:
!cd GFPGAN && pip install -r requirements.txt

In [ ]:
!cd GFPGAN && python setup.py develop

In [ ]:
!pip install realesrgan

In [ ]:
!cd GFPGAN && wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P experiments/pretrained_models

In [ ]:
!cd GFPGAN && python inference_gfpgan.py -i /content/GFPGAN/inputs/test2.jpg -o results/new -v 1.3 -s 2

In [ ]:
!pip install gfpgan

#FRAMER.py

In [ ]:
import sys
sys.path.append('/content/GFPGAN')
!cd GFPGAN

In [ ]:
!cd GFPGAN
import cv2
import os
from gfpgan import GFPGANer
import torch


def divide_video_into_frames(video_path, output_dir):
    #LOAD REALESRGAN
    upsample_bg = 'realesrgan'
    if upsample_bg == 'realesrgan':
        if not torch.cuda.is_available():  # CPU
            import warnings
            warnings.warn('The unoptimized RealESRGAN is slow on CPU. We do not use it. '
                          'If you really want to use it, please modify the corresponding codes.')
            bg_upsampler = None
        else:
            from basicsr.archs.rrdbnet_arch import RRDBNet
            from realesrgan import RealESRGANer
            model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=2)
            bg_upsampler = RealESRGANer(
                scale=2,
                model_path='https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth',
                model=model,
                tile=0,
                tile_pad=10,
                pre_pad=0,
                half=True)  # need to set False in CPU mode
    else:
        bg_upsampler = None

    #LOAD GFPGAN
    arch = 'clean'
    channel_multiplier = 2
    model_name = 'GFPGANv1.3'
    url = 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth'

    restorer = GFPGANer(
        model_path=url,
        upscale=2,
        arch=arch,
        channel_multiplier=channel_multiplier,
        bg_upsampler=bg_upsampler)

    # Get the video capture object
    video_capture = cv2.VideoCapture(video_path)

    # Get the number of frames in the video
    num_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

    # Iterate over the frames in the video
    for i in range(num_frames):
        # Read the frame from the video
        success, frame = video_capture.read()

        # If the frame was read successfully, save it
        if success:
            frame_name = os.path.join(output_dir, str(i) + ".jpg")
            cropped_faces, restored_faces, restored_img = restorer.enhance(
              frame,
              has_aligned=False,
              only_center_face=False,
              paste_back=True,
              weight=0.5)

            cv2.imwrite(frame_name, restored_img)

    # Close the video capture object
    video_capture.release()

if __name__ == "__main__":
    # Set the video path and output directory
    video_path = "/content/result_voice_theek.mp4"
    output_dir = "/content/Frames"

    # Divide the video into frames
    divide_video_into_frames(video_path, output_dir)